In [10]:
# Pandas
import pandas as pd
import talib as ta
import numpy as np
# from get_score import get_score_ch
from sqlalchemy import create_engine

engine= create_engine('sqlite:///../single_ticker_price/us/us_ticker_price_yf.db')
df = pd.read_sql('AAPL', engine)
df.rename(columns={'hgih':'high'},inplace=True)
df['chg_pct'] = ((df.close - df.open)/df.open)*100
df[df.chg_pct>0].chg_pct.describe()

count    4771.000000
mean        1.759120
std         1.744129
min         0.006525
25%         0.587720
50%         1.220762
75%         2.343926
max        19.265611
Name: chg_pct, dtype: float64

In [12]:
df = df[3600:]


In [13]:
df.index = range(len(df))

In [14]:
df

,open,high,low,close,Adj Close,volume,datetime,chg_pct
0,4.992500,5.127143,4.990714,5.087143,4.349600,439384400,2009-06-26,1.895706
1,5.123571,5.141071,5.055000,5.070357,4.335246,567616000,2009-06-29,-1.038613
2,5.092143,5.135714,5.064286,5.086786,4.349293,434224000,2009-06-30,-0.105206
3,5.125000,5.166429,5.090000,5.101071,4.361506,414178800,2009-07-01,-0.466910
4,5.044643,5.101071,4.992500,5.000714,4.275701,370479200,2009-07-02,-0.870807
...,...,...,...,...,...,...,...,...
3280,145.259995,147.550003,145.000000,147.039993,147.039993,64493200,2022-07-08,1.225388
3281,145.669998,146.639999,143.779999,144.869995,144.869995,63141600,2022-07-11,-0.549189
3282,145.759995,148.449997,145.050003,145.860001,145.860001,77588800,2022-07-12,0.068610
3283,142.990005,146.449997,142.119995,145.490005,145.490005,71185600,2022-07-13,1.748374


In [15]:
len_d = len(df)

In [25]:
df[10:20]

,open,high,low,close,Adj Close,volume,datetime,chg_pct
10,4.983571,5.083571,4.911786,5.083571,4.346545,483501200,2009-07-13,2.006591
11,5.072500,5.113571,5.041429,5.081071,4.344406,347247600,2009-07-14,0.168963
12,5.180000,5.250000,5.154286,5.245714,4.485180,485587200,2009-07-15,1.268617
13,5.205714,5.286429,5.198929,5.268571,4.504721,393570800,2009-07-16,1.207455
14,5.324286,5.429286,5.308214,5.419643,4.633893,602154000,2009-07-17,1.790981
15,5.473929,5.537143,5.388929,5.461071,4.669314,735526400,2009-07-20,-0.234894
16,5.474643,5.479643,5.348214,5.411071,4.626562,874781600,2009-07-21,-1.161216
17,5.635357,5.668929,5.575357,5.597857,4.786269,874104000,2009-07-22,-0.665440
18,5.593929,5.658571,5.555714,5.636429,4.819246,526962800,2009-07-23,0.759753
19,5.605357,5.714286,5.589286,5.713929,4.885513,438362400,2009-07-24,1.936933


In [33]:
len(df[10:20].query('chg_pct > 0')) / 10

0.7

In [40]:
df['raise_pct'] = 0
count = 0
for i in range(10,len_d):
    df.raise_pct.loc[i] = len(df[i-10:i].query('chg_pct > 0'))


/var/folders/lz/kp_y7_p51sbc8gskvwtst_6h0000gp/T/ipykernel_3976/3267708698.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['raise_pct'] = 0
/var/folders/lz/kp_y7_p51sbc8gskvwtst_6h0000gp/T/ipykernel_3976/3267708698.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.raise_pct.loc[i] = len(df[i-10:i].query('chg_pct > 0'))


In [44]:
len(df[df.raise_pct >= 7]) / len_d

0.19452054794520549

In [45]:
target_df = df[df.raise_pct >= 7]

In [47]:
target_df.datetime.to_csv('sold_datetime_tra2.csv',index=False)

In [48]:
datetime_df1 = pd.read_csv('get_profit_datetime.csv')
datetime_df2 = pd.read_csv('sold_datetime_tra2.csv')

In [56]:
merge_df = pd.merge(datetime_df1, datetime_df2, on=['datetime'], how='inner')
merge_df.drop(columns=['Unnamed: 0'],inplace=True)

In [57]:
merge_df.to_csv('result1.csv',index=None)

In [59]:
datetime_list = merge_df.datetime.to_list()

In [62]:
index_list = []
for item in datetime_list:
    temp_index = df[df['datetime']== item].index.values[0]
    # print(temp_index)
    index_list.append(temp_index)


In [63]:
index_list

[412,
 413,
 564,
 637,
 638,
 640,
 641,
 681,
 682,
 691,
 789,
 790,
 791,
 792,
 814,
 815,
 816,
 1233,
 1234,
 1235,
 1297,
 1298,
 1299,
 1300,
 1303,
 1344,
 1345,
 1346,
 1347,
 1348,
 1349,
 1350,
 1351,
 1352,
 1353,
 1358,
 1360,
 1361,
 1418,
 1420,
 1421,
 1909,
 1910,
 1915,
 1916,
 1918,
 1919,
 1920,
 1921,
 1924,
 1925,
 1926,
 1927,
 1928,
 1985,
 2032,
 2033,
 2034,
 2049,
 2050,
 2092,
 2107,
 2108,
 2110,
 2111,
 2112,
 2148,
 2216,
 2231,
 2232,
 2233,
 2234,
 2235,
 2236,
 2252,
 2254,
 2286,
 2595,
 2596,
 2597,
 2598,
 2599,
 2600,
 2601,
 2602,
 2609,
 2610,
 2611,
 2612,
 2613,
 2614,
 2615,
 2616,
 2617,
 2618,
 2632,
 2642,
 2643,
 2679,
 2680,
 2743,
 2753,
 2755,
 2762,
 2796,
 2804,
 2805,
 2806,
 2812,
 3030,
 3045,
 3046,
 3047,
 3048,
 3049,
 3062,
 3063,
 3147,
 3148,
 3149,
 3150,
 3151,
 3212,
 3215]

In [67]:
len_l = len(index_list)
for i in range(1,len_l):
    if index_list[i] - index_list[i-1] < 5:
        index_list.remove(index_list[i])
        len_l -= 1

In [69]:
index_list

[412,
 564,
 637,
 681,
 691,
 789,
 814,
 1233,
 1297,
 1303,
 1344,
 1352,
 1358,
 1418,
 1909,
 1915,
 1920,
 1926,
 1985,
 2032,
 2049,
 2092,
 2107,
 2112,
 2148,
 2216,
 2231,
 2252,
 2286,
 2595,
 2609,
 2617,
 2632,
 2642,
 2679,
 2743,
 2753,
 2762,
 2796,
 2804,
 2812,
 3030,
 3045,
 3062,
 3147,
 3212]

In [73]:
# test
len_l = len(index_list)
test_list = []
profit_list = []
non_profit_list = []
return_price_list = []
for l in range(len_l):
    i = index_list[l]
    price_now = df.close.loc[i]
    min_ = df.close.loc[i-90:i].min()
    max_ = df.close.loc[i:i+90].max()
    min__ = df.close.loc[i:i+90].min()
    # profit
    profit = (price_now - min_) / min_
    # non-profit
    non_profit = (max_ - price_now) / price_now
    # save
    return_price = (min__ - price_now) / price_now
    # test_list.append([profit, ])
    profit_list.append(profit)
    non_profit_list.append(non_profit)
    return_price_list.append(return_price)


In [83]:
len(index_list)

46

In [86]:
datetime_list = []
for item in index_list:
    datetime_list.append(df.datetime.loc[item])

In [87]:
len(datetime_list)

46

In [88]:
len(profit_list)

46

In [89]:
test_df = pd.DataFrame({
    'datetime':datetime_list,
    'profit':profit_list,
    'non_profit':non_profit_list, 
    'return_price':return_price_list
})

In [90]:
test_df

,datetime,profit,non_profit,return_price
0,2011-02-14,0.242021,0.010997,-0.122111
1,2011-09-21,0.307053,0.107585,-0.117848
2,2012-01-05,0.149792,0.521972,0.000000
3,2012-03-09,0.499491,0.167030,-0.027606
4,2012-03-23,0.639437,0.067410,-0.110612
5,2012-08-13,0.188410,0.114444,-0.190810
6,2012-09-18,0.324059,0.000271,-0.373310
7,2014-05-21,0.213154,0.192624,0.000000
8,2014-08-21,0.359294,0.183138,-0.042951
9,2014-08-29,0.367318,0.160976,-0.060878
